# Python performance tips and best practice for CLIMADA developers

This guide covers the following recommendations:

⏲️ **Use profiling tools** to find and assess performance bottlenecks.<br />
🔁 **Replace for-loops** by built-in functions and efficient external implementations.<br />
📝 **Consider algorithmic performance**, not only implementation performance.<br />
🧊 **Get familiar with NumPy:** vectorized functions, slicing, masks and broadcasting.<br />
⚫ **Miscellaneous:** sparse arrays, Numba, parallelization, huge files (xarray), memory.<br />
⚠️ **Don't over-optimize** at the expense of readability and usability.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-Profiling" data-toc-modified-id="1-Profiling-1">1 Profiling</a></span></li><li><span><a href="#2-General-considerations" data-toc-modified-id="2-General-considerations-2">2 General considerations</a></span><ul class="toc-item"><li><span><a href="#2.1-for-loops" data-toc-modified-id="2.1-for-loops-2.1">2.1 for-loops</a></span></li><li><span><a href="#2.2-Converting-data-structures" data-toc-modified-id="2.2-Converting-data-structures-2.2">2.2 Converting data structures</a></span></li><li><span><a href="#2.3-Always-consider-several-implementations" data-toc-modified-id="2.3-Always-consider-several-implementations-2.3">2.3 Always consider several implementations</a></span></li><li><span><a href="#2.4-Efficient-algorithms" data-toc-modified-id="2.4-Efficient-algorithms-2.4">2.4 Efficient algorithms</a></span></li><li><span><a href="#2.5-Memory-usage" data-toc-modified-id="2.5-Memory-usage-2.5">2.5 Memory usage</a></span></li></ul></li><li><span><a href="#3-NumPy-related-tips-and-best-practice" data-toc-modified-id="3-NumPy-related-tips-and-best-practice-3">3 NumPy-related tips and best practice</a></span><ul class="toc-item"><li><span><a href="#3.1-Vectorized-functions" data-toc-modified-id="3.1-Vectorized-functions-3.1">3.1 Vectorized functions</a></span></li><li><span><a href="#3.2-Broadcasting" data-toc-modified-id="3.2-Broadcasting-3.2">3.2 Broadcasting</a></span></li><li><span><a href="#3.3-A-note-on-in-place-operations" data-toc-modified-id="3.3-A-note-on-in-place-operations-3.3">3.3 A note on in-place operations</a></span></li></ul></li><li><span><a href="#4-Miscellaneous" data-toc-modified-id="4-Miscellaneous-4">4 Miscellaneous</a></span><ul class="toc-item"><li><span><a href="#4.1-Sparse-matrices" data-toc-modified-id="4.1-Sparse-matrices-4.1">4.1 Sparse matrices</a></span></li><li><span><a href="#4.2-Fast-for-loops-using-Numba" data-toc-modified-id="4.2-Fast-for-loops-using-Numba-4.2">4.2 Fast for-loops using Numba</a></span></li><li><span><a href="#4.3-Parallelizing-tasks" data-toc-modified-id="4.3-Parallelizing-tasks-4.3">4.3 Parallelizing tasks</a></span></li><li><span><a href="#4.4-Read-NetCDF-datasets-with-xarray" data-toc-modified-id="4.4-Read-NetCDF-datasets-with-xarray-4.4">4.4 Read NetCDF datasets with <code>xarray</code></a></span></li></ul></li><li><span><a href="#5-Take-home-messages" data-toc-modified-id="5-Take-home-messages-5">5 Take-home messages</a></span></li></ul></div>

## 1 Profiling
Python comes with powerful packages for the **performance assessment** of your code. Within IPython and notebooks, there are several magic commands for this task:

* `%time`: Time the execution of a single statement
* `%timeit`: Time repeated execution of a single statement for more accuracy
* `%%timeit` Does the same as `%timeit` for a whole cell
* `%prun`: Run code with the profiler
* `%lprun`: Run code with the line-by-line profiler
* `%memit`: Measure the memory use of a single statement
* `%mprun`: Run code with the line-by-line memory profiler

More information on profiling in the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html).

Also useful: unofficial Jupyter extension [Execute Time](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/execute_time/readme.html).

While it's easy to assess how fast or slow parts of your code are, including finding the bottlenecks, **generating an improved version of it is much harder**. This guide is about **simple best practices** that everyone should know who works with Python, especially when models are performance-critical.

In the following, we will **focus on arithmetic operations** because they play an important role in CLIMADA. Operations on non-numeric objects like strings, graphs, databases, file or network IO might be just as relevant inside and outside of the CLIMADA context. Some of the tips presented here do also apply to other contexts, but **it's always worth looking for context-specific performance guides**.

## 2 General considerations
This section will be concerned with:

🔁 **for-loops** and built-ins<br />
📦 **external implementations** and converting data structures<br />
📝 **algorithmic efficiency**<br />
💾 **memory usage**

𝚺 As this section's toy example, let's assume we want to sum up all the numbers in a list:

In [ ]:
list_of_numbers = list(range(10000))

### 2.1 for-loops

A developer with a background in C++ would probably loop over the entries of the list:

In [2]:
%%timeit
result = 0
for i in list_of_numbers:
    result += i

332 µs ± 65.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The built-in function `sum` is much faster:

In [3]:
%timeit sum(list_of_numbers)

54.9 µs ± 5.63 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


The timing improves by a factor of 5-6 and this is not a coincidence: **for-loops generally tend to get prohibitively expensive** when the number of iterations increases.

**💡 When you have a for-loop with many iterations in your code, check for built-in functions or efficient external implementations of your programming task.**

A special case worth noting are `append` operations on lists which can often be replaced by more efficient *list comprehensions*.

### 2.2 Converting data structures

💡 **When you find an external library that solves your task efficiently, always consider that it might be necessary to convert your data structure which takes time.**

For arithmetic operations, NumPy is a great library, but if your data comes as a Python list, NumPy will spend quite some time converting it to a NumPy array:

In [4]:
import numpy as np
%timeit np.sum(list_of_numbers)

572 µs ± 80 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


This operation is even slower than the for-loop!

However, if you can somehow obtain your data in the form of **NumPy arrays from the start**, or if you perform many operations that might compensate for the conversion time, the gain in performance can be considerable:

In [5]:
# do the conversion outside of the `%timeit`
ndarray_of_numbers = np.array(list_of_numbers)
%timeit np.sum(ndarray_of_numbers)

10.6 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Indeed, this is 5-6 times faster than the built-in `sum` and 20-30 times faster than the for-loop.

### 2.3 Always consider several implementations

Even for such a basic task as summing, there exist several implementations whose performance can vary more than you might expect:

In [6]:
%timeit ndarray_of_numbers.sum()
%timeit np.einsum("i->", ndarray_of_numbers)

9.07 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
5.55 µs ± 383 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


This is up to 50 times faster than the for-loop. More information about the `einsum` function will be given in the NumPy section of this guide.

### 2.4 Efficient algorithms

💡 **Consider algorithmic performance, not only implementation performance.**

All of the examples above do exactly the same thing, algorithmically. However, often the largest performance improvements can be obtained from **algorithmic changes**. This is the case when your model or your data contain symmetries or more complex structure that allows you to skip or boil down arithmetic operations.

In our example, we are summing the numbers from 1 to 10,000 and it's a well known mathematical theorem that this can be done using only two multiplications and an increment:

In [7]:
n = max(list_of_numbers)
%timeit 0.5 * n * (n + 1)

83.1 ns ± 2.5 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Not surprisingly, This is almost 100 times faster than even the fastest implementation of the 10,000 summing operations listed above.

You don't need a degree in maths to find algorithmic improvements. Other algorithmic improvements that are often easy to detect are:
* **Filter your data set as much as possible** to perform operations only on those entries that are really relevant. <br> ***Example:*** When computing a physical hazard (e.g. extreme wind) with CLIMADA, restrict to Centroids on land unless you know that some of your exposure is off shore.
* Make sure to **detect inconsistent or trivial input parameters early on**, before starting any operations. <br> ***Example:*** If your code does some complicated stuff and applies a user-provided normalization factor at the very end, make sure to check that the factor is not 0 before you start applying those complicated operations.

📝 **In general: Before starting to code, take pen and paper and write down what you want to do from an algorithmic perspective.**

### 2.5 Memory usage
💡 **Be careful with deep copies of large data sets and only load portions of large files into memory as needed.**

Write your code in such a way that you **handle large amounts of data chunk by chunk** so that Python does not need to load everything into memory before performing any operations. When you do, Python's [generators](https://wiki.python.org/moin/Generators) might help you with the implementation.

🐌 Allocating unnecessary amounts of memory might slow down your code substantially due to swapping.

## 3 NumPy-related tips and best practice

As mentioned above, arithmetic operations in Python can profit a lot from NumPy's capabilities. In this section, we collect some tips how to make use of NumPy's capabilities when performance is an issue.

### 3.1 Vectorized functions
We mentioned above that Python's **for-loops are really slow**. This is even more important when looping over the entries in a NumPy array. Fortunately, NumPy's masks, slicing notation and vectorization capabilities help to avoid for-loops in almost every possible situation:

In [8]:
# TASK: compute the column-sum of a 2-dimensional array
input_arr = np.random.rand(100, 3)

In [9]:
%%timeit
# SLOW: summing over columns using loops
output = np.zeros(100)
for row_i in range(input_arr.shape[0]):
    for col_i in range(input_arr.shape[1]):
        output[row_i] += input_arr[row_i, col_i]

145 µs ± 5.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
# FASTER: using NumPy's vectorized `sum` function with `axis` attribute
%timeit output = input_arr.sum(axis=1)

4.23 µs ± 216 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In the special case of multiplications and sums (linear operations) over the axes of two multi-dimensional arrays, NumPy's **`einsum`** is even faster:

In [11]:
%timeit output = np.einsum("ij->i", input_arr)

2.38 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Another `einsum` example: **Euclidean norms**

In [12]:
many_vectors = np.random.rand(1000, 3)
%timeit np.sqrt((many_vectors**2).sum(axis=1))
%timeit np.linalg.norm(many_vectors, axis=1)
%timeit np.sqrt(np.einsum("...j,...j->...", many_vectors, many_vectors))

24.4 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
26.5 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
9.5 µs ± 91.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


For more information about the capabilities of NumPy's `einsum` function, refer to [the official NumPy documentation](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html). However, note that future releases of NumPy will eventually improve the performance of core functions, so that `einsum` will become an example of over-optimization (see above) at some point. Whenever you use `einsum`, consider adding a comment that explains what it does for users that are not familiar with `einsum`'s syntax.

Not only `sum`, but many NumPy functions come with similar vectorization capabilities. You can take minima, maxima, means or standard deviations along selected axes. But did you know that the same is true for the `diff` and `argmin` functions?

In [13]:
arr = np.random.randint(low=0, high=10, size=(4, 3))
arr

array([[4, 2, 6],
       [2, 3, 4],
       [3, 3, 3],
       [3, 2, 4]])

In [14]:
arr.argmin(axis=1)

array([1, 0, 0, 1])

### 3.2 Broadcasting

When operations are performed on several arrays, possibly of differing shapes, be sure to use NumPy's **broadcasting** capabilities. This will save you a lot of memory and time when performing arithmetic operations.

***Example:*** We want to multiply the columns of a two-dimensional array by values stored in a one-dimensional array. There are two naive approaches to this:

In [15]:
input_arr = np.random.rand(100, 3)
col_factors = np.random.rand(3)

In [16]:
# SLOW: stack/tile the one-dimensional array to be two-dimensional
%timeit output = np.tile(col_factors, (input_arr.shape[0], 1)) * input_arr

5.67 µs ± 718 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
%%timeit
# SLOW: loop over columns and factors
output = input_arr.copy()
for i, factor in enumerate(col_factors):
    output[:, i] *= factor

9.63 µs ± 95.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


The idea of *broadcasting* is that NumPy **automatically matches axes from right to left and implicitly repeats data along missing axes** if necessary:

In [18]:
%timeit output = col_factors * input_arr

1.41 µs ± 51.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


For automatic broadcasting, the *trailing* dimensions of two arrays have to match. NumPy is matching the shapes of the arrays *from right to left*. If you happen to have arrays where other dimensions match, **you have to tell NumPy which dimensions to add by adding an axis of length 1 for each missing dimension**:

In [19]:
input_arr = np.random.rand(3, 100)
row_factors = np.random.rand(3)
output = row_factors.reshape(3, 1) * input_arr

Because this concept is so important, there is a short-hand notation for adding an axis of length 1. In the slicing notation, **add `None` in those positions where broadcasting should take place**.

In [20]:
input_arr = np.random.rand(3, 100)
row_factors = np.random.rand(3)
output = row_factors[:, None] * input_arr

In [21]:
input_arr = np.random.rand(7, 3, 5, 4, 6)
factors = np.random.rand(7, 3, 4)
output = factors[:, :, None, :, None] * input_arr

### 3.3 A note on in-place operations
While **in-place operations** are generally faster than long and explicit expressions, they shouldn't be over-estimated when looking for performance bottlenecks. Often, the loss in code readability is not justified because NumPy's memory management is really fast.

💡 **Don't over-optimize!**

In [22]:
shape = (1200, 1700)
arr_a = np.random.rand(*shape)
arr_b = np.random.rand(*shape)
arr_c = np.random.rand(*shape)

In [23]:
# long expression in one line
%timeit arr_d = arr_c * (arr_a + arr_b) - arr_a + arr_c

17.3 ms ± 820 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%%timeit
# almost same performance: in-place operations
arr_d = arr_a + arr_b
arr_d *= arr_c
arr_d -= arr_a
arr_d += arr_c

17.4 ms ± 618 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
# You may want to install the module "memory_profiler" first: activate the environment climada_env in an Anaconda prompt,
# type "pip install memory_profiler" and execute it
%load_ext memory_profiler

In [26]:
# long expression in one line
%memit arr_d = arr_c * (arr_a + arr_b) - arr_a + arr_c

peak memory: 156.68 MiB, increment: 31.20 MiB


In [27]:
%%memit
# almost same memory usage: in-place operations
arr_d = arr_a + arr_b
arr_d *= arr_c
arr_d -= arr_a
arr_d += arr_c

peak memory: 157.27 MiB, increment: 0.00 MiB


## 4 Miscellaneous

### 4.1 Sparse matrices

In many contexts, we deal with sparse matrices or sparse data structures, i.e. two-dimensional arrays where most of the entries are 0. In CLIMADA, this is especially the case for the `intensity` attributes of `Hazard` objects. This kind of data is usually handled using SciPy's submodule [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html).

💡 **When dealing with sparse matrices make sure that you always understand exactly which of your variables are sparse and which are dense and only switch from sparse to dense when absolutely necessary.**

💡 **Multiplications (`multiply`) and matrix multiplications (`dot`) are often faster than operations that involve masks or indexing.**

As an example for the last rule, consider the problem of multiplying certain rows of a sparse array by a scalar:

In [28]:
import scipy.sparse as sparse

array = np.tile(np.array([0, 0, 0, 2, 0, 0, 0, 1, 0], dtype=np.float64), (100, 80))
row_mask = np.tile(np.array([False, False, True, False, True], dtype=bool), (20,))

In the following cells, note that the code in the first line after the `%%timeit` statement is not timed, it's the setup line.

In [29]:
%%timeit sparse_array = sparse.csr_matrix(array)
sparse_array[row_mask, :] *= 5

/home/tovogt/.local/share/miniconda3/envs/tc/lib/python3.7/site-packages/scipy/sparse/data.py:55: RuntimeWarning: overflow encountered in multiply
  self.data *= other


1.52 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
%%timeit sparse_array = sparse.csr_matrix(array)
sparse_array.multiply(np.where(row_mask, 5, 1)[:, None]).tocsr()

340 µs ± 7.32 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
%%timeit sparse_array = sparse.csr_matrix(array)
sparse.diags(np.where(row_mask, 5, 1)).dot(sparse_array)

400 µs ± 6.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 4.2 Fast for-loops using Numba
As a last resort, if there's no way to avoid a for-loop even with NumPy's vectorization capabilities, you can use the `@njit` decorator provided by the Numba package:

In [32]:
from numba import njit

@njit
def sum_array(arr):
    result = 0.0
    for i in range(arr.shape[0]):
        result += arr[i]
    return result

In fact, the Numba function is more than 100 times faster than without the decorator:

In [33]:
input_arr = np.float64(np.random.randint(low=0, high=10, size=(10000,)))

In [34]:
%timeit sum_array(input_arr)

10.9 µs ± 444 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [35]:
# Call the function without the @njit
%timeit sum_array.py_func(input_arr)

1.84 ms ± 65.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


However, whenever available, **NumPy's own vectorized functions will usually be faster than Numba**.

In [36]:
%timeit np.sum(input_arr)
%timeit input_arr.sum()
%timeit np.einsum("i->", input_arr)

7.6 µs ± 687 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
5.27 µs ± 411 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.89 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


💡 **Make sure you understand the basic idea behind Numba before using it, read the [Numba docs](https://numba.readthedocs.io/en/stable/user/5minguide.html).**

💡 **Don't use `@jit`, but use `@njit` which is an alias for `@jit(nopython=True)`.**

When you know what you are doing, the `fastmath` and `parallel` options can boost performance even further: read more about this in the [Numba docs](https://numba.pydata.org/numba-doc/dev/user/performance-tips.html).

### 4.3 Parallelizing tasks

Depending on your hardware setup, parallelizing tasks using [pathos](https://pathos.readthedocs.io/en/latest/pathos.html) and [Numba's automatic parallelization feature](https://numba.readthedocs.io/en/stable/user/parallel.html) can improve the performance of your implementation.

💡 **Expensive hardware is no excuse for inefficient code.**

Many tasks in CLIMADA could profit from GPU implementations. However, currently there are **no plans to include GPU support in CLIMADA** because of the considerable development and maintenance workload that would come with it. If you want to change this, contact the core team of developers, open an issue or mention it in the bi-weekly meetings.

### 4.4 Read NetCDF datasets with `xarray`
When dealing with NetCDF datasets, memory is often an issue, because even if the file is only a few megabytes in size, the uncompressed raw arrays contained within can be several gigabytes large (especially when data is sparse or similarly structured). One way of dealing with this situation is to open the dataset with `xarray`.

💡 **`xarray` allows to read the shape and type of variables contained in the dataset without loading any of the actual data into memory.**

Furthermore, when loading slices and arithmetically aggregating variables, memory is allocated not more than necessary, but values are obtained on-the-fly from the file.

## 5 Take-home messages

We conclude by repeating the gist of this guide:

⏲️ **Use profiling tools** to find and assess performance bottlenecks.<br />
🔁 **Replace for-loops** by built-in functions and efficient external implementations.<br />
📝 **Consider algorithmic performance**, not only implementation performance.<br />
🧊 **Get familiar with NumPy:** vectorized functions, slicing, masks and broadcasting.<br />
⚫ **Miscellaneous:** sparse arrays, Numba, parallelization, huge files (xarray), memory.<br />
⚠️ **Don't over-optimize** at the expense of readability and usability.